In [119]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchaudio-2.8.0-cp311-cp311-win_amd64.whl.metadata (7.2 kB)
  Using cached torchvision-0.23.0-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached transformers-4.39.0-py3-none-any.whl.metadata (134 kB)
  Using cached datasets-4.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached trl-0.23.1-py3-none-any.whl.metadata (11 kB)
  Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl.metadata (6.8 kB)
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_huggingface-0.3.0-py3-none-any.whl.metadata (996 bytes)
  Using cached langchain_huggingface-0.2.0-py3-none-any.whl.metadata (941 bytes)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached langchai

error: resolution-too-deep

× Dependency resolution exceeded maximum depth
╰─> Pip cannot resolve the current dependencies as the dependency graph is too complex for pip to solve efficiently.

hint: Try adding lower bounds to constrain your dependencies, for example: 'package>=2.0.0' instead of just 'package'.

Link: https://pip.pypa.io/en/stable/topics/dependency-resolution/#handling-resolution-too-deep-errors


In [92]:
import os
import shutil
import sqlite3

import pandas as pd
import requests

db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file = "travel2.sqlite"
# The backup lets us restart for each tutorial section
backup_file = "travel2.backup.sqlite"
overwrite = False
if overwrite or not os.path.exists(local_file):
    response = requests.get(db_url)
    response.raise_for_status()  # Ensure the request was successful
    with open(local_file, "wb") as f:
        f.write(response.content)
    # Backup - we will use this to "reset" our DB in each section
    shutil.copy(local_file, backup_file)

In [ ]:
# Convert the flights to present time for our tutorial
def update_dates(file):
    shutil.copy(backup_file, file)
    conn = sqlite3.connect(file)
    cursor = conn.cursor()

    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    ).name.tolist()
    tdf = {}
    for t in tables:
        tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

    example_time = pd.to_datetime(
        tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
    ).max()
    current_time = pd.to_datetime("now").tz_localize(example_time.tz)
    time_diff = current_time - example_time

    tdf["bookings"]["book_date"] = (
        pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
        + time_diff
    )

    datetime_columns = [
        "scheduled_departure",
        "scheduled_arrival",
        "actual_departure",
        "actual_arrival",
    ]
    for column in datetime_columns:
        tdf["flights"][column] = (
            pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT)) + time_diff
        )

    for table_name, df in tdf.items():
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    del df
    del tdf
    conn.commit()

    return file


db = update_dates(local_file)

In [80]:
db

'travel2.sqlite'

In [81]:
conn = sqlite3.connect(local_file)
cursor = conn.cursor()

tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    ).name.tolist()

In [82]:
tables

['aircrafts_data',
 'airports_data',
 'boarding_passes',
 'bookings',
 'flights',
 'seats',
 'ticket_flights',
 'tickets',
 'car_rentals',
 'hotels',
 'trip_recommendations']

In [131]:
pd.read_sql(f"SELECT * from flights WHERE departure_airport = 'JFK' AND status = 'Scheduled'", conn)

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,21314,TP0020,2025-10-20 18:35:25.335891-04:00,2025-10-20 20:45:25.335891-04:00,JFK,BSL,Scheduled,CR2,None,None
1,21315,TP0020,2025-10-25 18:35:25.335891-04:00,2025-10-25 20:45:25.335891-04:00,JFK,BSL,Scheduled,CR2,None,None
2,21316,TP0020,2025-11-10 18:35:25.335891-04:00,2025-11-10 20:45:25.335891-04:00,JFK,BSL,Scheduled,CR2,None,None
3,21321,TP0020,2025-10-29 18:35:25.335891-04:00,2025-10-29 20:45:25.335891-04:00,JFK,BSL,Scheduled,CR2,None,None
4,21322,TP0020,2025-11-08 18:35:25.335891-04:00,2025-11-08 20:45:25.335891-04:00,JFK,BSL,Scheduled,CR2,None,None
...,...,...,...,...,...,...,...,...,...,...
66,21456,KL0134,2025-10-24 14:20:25.335891-04:00,2025-10-24 15:15:25.335891-04:00,JFK,ARN,Scheduled,CR2,None,None
67,21465,KL0134,2025-10-29 14:20:25.335891-04:00,2025-10-29 15:15:25.335891-04:00,JFK,ARN,Scheduled,CR2,None,None
68,21466,KL0134,2025-11-05 14:20:25.335891-04:00,2025-11-05 15:15:25.335891-04:00,JFK,ARN,Scheduled,CR2,None,None
69,21467,KL0134,2025-11-01 14:20:25.335891-04:00,2025-11-01 15:15:25.335891-04:00,JFK,ARN,Scheduled,CR2,None,None


In [137]:
pd.read_sql(f"SELECT sum(total_amount) FROM bookings WHERE book_date LIKE '2025-10-01%'",conn)

,sum(total_amount)
0,443804500


In [143]:
pd.read_sql(f"SELECT departure_airport, arrival_airport, COUNT(*) AS total_scheduled_flights FROM flights WHERE status = 'Scheduled' GROUP BY departure_airport, arrival_airport ORDER BY total_scheduled_flights DESC LIMIT 10",conn)

,departure_airport,arrival_airport,total_scheduled_flights
0,OSL,HAM,147
1,HAM,OSL,146
2,BSL,HAM,117
3,HAM,BSL,117
4,BNE,OSL,89
5,BNE,BSL,88
6,BNE,SHA,87
7,HAM,SHA,87
8,OSL,BNE,87
9,SHA,BNE,87


In [142]:
pd.read_sql(f"SELECT departure_airport, arrival_airport, COUNT(*) AS total_scheduled_flights FROM flights WHERE status = 'Scheduled' GROUP BY departure_airport, arrival_airport ORDER BY total_scheduled_flights DESC", conn)

,departure_airport,arrival_airport,total_scheduled_flights
0,OSL,HAM,147
1,HAM,OSL,146
2,BSL,HAM,117
3,HAM,BSL,117
4,BNE,OSL,89
...,...,...,...
613,SAW,OSL,3
614,SHA,LAD,3
615,SHA,LGW,3
616,SLC,PNH,3


In [136]:
pd.read_sql(f"SELECT * FROM bookings where book_date between '2025-10-01%' and '2025-10-03%'",conn)

,book_ref,book_date,total_amount
0,0006C3,2025-10-02 15:47:25.335891+00:00,59400
1,000F7D,2025-10-02 13:23:25.335891+00:00,20600
2,0018AB,2025-10-03 07:34:25.335891+00:00,277600
3,001A4A,2025-10-02 08:36:25.335891+00:00,35900
4,001F27,2025-10-03 23:15:25.335891+00:00,39700
...,...,...,...
11117,FFE6D2,2025-10-02 08:47:25.335891+00:00,123800
11118,FFEFBB,2025-10-02 05:41:25.335891+00:00,56000
11119,FFF1FD,2025-10-02 01:10:25.335891+00:00,82900
11120,FFF840,2025-10-02 21:21:25.335891+00:00,43600


In [ ]:
pd.read_sql(f"SELECT * FROM airports_data",conn)

,airport_code,airport_name,city,coordinates,timezone
0,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,"[33.6407, -84.4277]",America/New_York
1,PEK,Beijing Capital International Airport,Beijing,"[40.0799, 116.6031]",Asia/Shanghai
2,DXB,Dubai International Airport,Dubai,"[25.2532, 55.3657]",Asia/Dubai
3,LAX,Los Angeles International Airport,Los Angeles,"[33.9416, -118.4085]",America/Los_Angeles
4,HND,Tokyo Haneda Airport,Tokyo,"[35.5494, 139.7798]",Asia/Tokyo
...,...,...,...,...,...
110,JRO,Kilimanjaro International Airport,Arusha,"[-3.4294, 37.0741]",Africa/Dar_es_Salaam
111,SEZ,Seychelles International Airport,Victoria,"[-4.6717, 55.5122]",Indian/Mahe
112,HRE,Robert Gabriel Mugabe International Airport,Harare,"[-17.9319, 31.0929]",Africa/Harare
113,LAD,Quatro de Fevereiro International Airport,Luanda,"[-8.8575, 13.2306]",Africa/Luanda


In [123]:
pd.read_sql(f"SELECT sum(total_amount) FROM bookings WHERE book_date like '2025-10-01%'", conn)

,sum(total_amount)
0,443804500


In [121]:
pd.read_sql(f"SELECT * FROM bookings", conn)

,book_ref,book_date,total_amount
0,00000F,2025-09-04 07:17:25.335891+00:00,265700
1,000012,2025-09-13 13:07:25.335891+00:00,37900
2,000068,2025-10-15 18:32:25.335891+00:00,18100
3,000181,2025-10-10 17:33:25.335891+00:00,131800
4,0002D8,2025-10-08 01:45:25.335891+00:00,23600
...,...,...,...
262783,FFFEF3,2025-09-16 11:28:25.335891+00:00,56000
262784,FFFF2C,2025-10-08 10:00:25.335891+00:00,10800
262785,FFFF43,2025-09-20 00:47:25.335891+00:00,78500
262786,FFFFA8,2025-10-08 08:50:25.335891+00:00,28800


In [125]:
pd.read_sql(f"SELECT * from tickets", conn)

,ticket_no,book_ref,passenger_id
0,9880005432000987,06B046,8149 604011
1,9880005432000988,06B046,8499 420203
2,9880005432000989,E170C3,1011 752484
3,9880005432000990,E170C3,4849 400049
4,9880005432000991,F313DD,6615 976589
...,...,...,...
366728,1760005435999869,D730BA,0474 690760
366729,1760005435999870,D730BA,6535 751108
366730,1760005435999871,A1AD46,1596 156448
366731,0060005435999872,7B6A53,9374 822707


In [134]:
pd.read_sql(f"SELECT model, range FROM aircrafts_data WHERE range > 5000", conn)


,model,range
0,Boeing 777-300,11100
1,Boeing 767-300,7900
2,Airbus A320-200,5700
3,Airbus A321-200,5600
4,Airbus A319-100,6700


In [128]:
pd.read_sql(f"SELECT * from aircrafts_data", conn)

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
2,SU9,Sukhoi Superjet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
6,733,Boeing 737-300,4200
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


In [127]:
pd.read_sql(f"SELECT * from seats", conn)

,aircraft_code,seat_no,fare_conditions
0,319,2A,Business
1,319,2C,Business
2,319,2D,Business
3,319,2F,Business
4,319,3A,Business
...,...,...,...
1334,773,48H,Economy
1335,773,48K,Economy
1336,773,49A,Economy
1337,773,49C,Economy


In [126]:
pd.read_sql(f"SELECT * from ticket_flights", conn)

,ticket_no,flight_id,fare_conditions,amount
0,0060005432159776,30625,Business,42100
1,0060005435212351,30625,Business,42100
2,0060005435212386,30625,Business,42100
3,0060005435212381,30625,Business,42100
4,0060005432211370,30625,Business,42100
...,...,...,...,...
1045721,2350005435097522,32094,Economy,5200
1045722,2350005435097521,32094,Economy,5200
1045723,2200005435104384,32094,Economy,5200
1045724,2200005435104352,32094,Economy,5200


In [124]:
pd.read_sql(f"SELECT * from car_rentals", conn)

,id,name,location,price_tier,start_date,end_date,booked
0,1,Europcar,Basel,Economy,2024-04-14,2024-04-11,1
1,2,Avis,Basel,Luxury,2024-04-10,2024-04-20,1
2,3,Hertz,Zurich,Midsize,2024-04-10,2024-04-07,1
3,4,Sixt,Bern,SUV,2024-04-20,2024-04-26,1
4,5,Budget,Lucerne,Compact,2024-04-01,2024-04-19,1
5,6,Alamo,Geneva,Full-size,2024-04-27,2024-04-18,1
6,7,Enterprise,Basel,Premium,2024-04-22,2024-04-20,0
7,8,National,Zurich,Economy,2024-04-08,2024-04-21,0
8,9,Thrifty,Basel,Midsize,2024-04-17,2024-04-26,0
9,10,Dollar,Bern,Compact,2024-04-23,2024-04-03,0


In [84]:
tdf = {}
for t in tables:
        tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

In [85]:
pd.read_sql(f"SELECT * from airports_data", conn)

,airport_code,airport_name,city,coordinates,timezone
0,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,"[33.6407, -84.4277]",America/New_York
1,PEK,Beijing Capital International Airport,Beijing,"[40.0799, 116.6031]",Asia/Shanghai
2,DXB,Dubai International Airport,Dubai,"[25.2532, 55.3657]",Asia/Dubai
3,LAX,Los Angeles International Airport,Los Angeles,"[33.9416, -118.4085]",America/Los_Angeles
4,HND,Tokyo Haneda Airport,Tokyo,"[35.5494, 139.7798]",Asia/Tokyo
...,...,...,...,...,...
110,JRO,Kilimanjaro International Airport,Arusha,"[-3.4294, 37.0741]",Africa/Dar_es_Salaam
111,SEZ,Seychelles International Airport,Victoria,"[-4.6717, 55.5122]",Indian/Mahe
112,HRE,Robert Gabriel Mugabe International Airport,Harare,"[-17.9319, 31.0929]",Africa/Harare
113,LAD,Quatro de Fevereiro International Airport,Luanda,"[-8.8575, 13.2306]",Africa/Luanda


In [86]:
pd.read_sql(f"SELECT * from boarding_passes", conn)

,ticket_no,flight_id,boarding_no,seat_no
0,0060005435212351,30625,1,2D
1,0060005435212386,30625,2,3G
2,0060005435212381,30625,3,4H
3,0060005432211370,30625,4,5D
4,0060005435212357,30625,5,11A
...,...,...,...,...
579681,0060005434302871,19945,85,20F
579682,7840005432892791,19945,86,21C
579683,0060005434302869,19945,87,20E
579684,1310005432802476,19945,88,21F


In [ ]:
import sqlite3

conn = sqlite3.connect("travel2.sqlite")
cursor = conn.cursor()

# Correct way to update
cursor.execute("UPDATE hotels SET booked = 1 WHERE id in(1,2,3,4,5,6)")
cursor.execute("UPDATE trip_recommendations SET booked = 1 WHERE id in(1,2,3,4,5,6)")
cursor.execute("UPDATE car_rentals SET booked = 1 WHERE id in(1,2,3,4,5,6)")
# Commit changes
conn.commit()




In [105]:
pd.read_sql(f"SELECT * FROM hotels where booked=1", conn)

,id,name,location,price_tier,checkin_date,checkout_date,booked
0,1,Hilton Basel,Basel,Luxury,2024-04-22,2024-04-20,1
1,2,Marriott Zurich,Zurich,Upscale,2024-04-14,2024-04-21,1
2,3,Hyatt Regency Basel,Basel,Upper Upscale,2024-04-02,2024-04-20,1
3,4,Radisson Blu Lucerne,Lucerne,Midscale,2024-04-24,2024-04-05,1
4,5,Best Western Bern,Bern,Upper Midscale,2024-04-23,2024-04-01,1
5,6,InterContinental Geneva,Geneva,Luxury,2024-04-23,2024-04-28,1


In [101]:
pd.read_sql(f"SELECT T1.id, T1.name FROM hotels AS T1 JOIN trip_recommendations AS T2 ON T1.id = T2.id WHERE T2.location = 'Basel'", conn)

,id,name
0,1,Hilton Basel
1,2,Marriott Zurich
2,8,Holiday Inn Basel


In [107]:
pd.read_sql(f"SELECT location, COUNT(*) AS total_booked_hotels FROM hotels WHERE booked = 1 GROUP BY location HAVING COUNT(*) >= 2", conn)

,location,total_booked_hotels
0,Basel,2


In [111]:
pd.read_sql(f"SELECT T1.name FROM hotels AS T1 JOIN trip_recommendations AS T2 ON T1.id = T2.id WHERE T2.location = 'Basel'",conn)

,name
0,Hilton Basel
1,Marriott Zurich
2,Holiday Inn Basel


In [112]:
pd.read_sql(f"SELECT T1.name FROM hotels AS T1 JOIN trip_recommendations AS T2 ON T1.id = T2.id WHERE T2.location = 'Zurich'",conn)

,name
0,Hyatt Regency Basel
1,Sheraton Zurich


In [96]:
pd.read_sql(f"SELECT h.name AS hotel_name, h.location, t.name AS recommendation_name, t.keywords, t.details FROM hotels h JOIN trip_recommendations t ON h.location = t.location WHERE h.booked = 1", conn)

,hotel_name,location,recommendation_name,keywords,details
0,Hilton Basel,Basel,Basel Minster,"landmark, history","Visit the historic Basel Minster, a beautiful ..."
1,Hilton Basel,Basel,Basel Zoo,"wildlife, zoo",Spend a day exploring the diverse animal exhib...
2,Hilton Basel,Basel,Kunstmuseum Basel,"art, museum",Explore the extensive art collection at the Ku...
3,Marriott Zurich,Zurich,Swiss National Museum,"history, museum",Learn about Swiss history and culture at the S...
4,Marriott Zurich,Zurich,Zurich Old Town,"history, architecture",Take a stroll through the charming streets of ...
5,Hyatt Regency Basel,Basel,Basel Minster,"landmark, history","Visit the historic Basel Minster, a beautiful ..."
6,Hyatt Regency Basel,Basel,Basel Zoo,"wildlife, zoo",Spend a day exploring the diverse animal exhib...
7,Hyatt Regency Basel,Basel,Kunstmuseum Basel,"art, museum",Explore the extensive art collection at the Ku...
8,Radisson Blu Lucerne,Lucerne,Lucerne Chapel Bridge,"landmark, history",Walk across the iconic Chapel Bridge in Lucerne.
9,Radisson Blu Lucerne,Lucerne,Mount Pilatus,"nature, mountain",Take a cable car ride to the top of Mount Pila...


In [90]:
pd.read_sql(f"SELECT * from bookings", conn)

,book_ref,book_date,total_amount
0,00000F,2025-09-04 02:28:23.316556+00:00,265700
1,000012,2025-09-13 08:18:23.316556+00:00,37900
2,000068,2025-10-15 13:43:23.316556+00:00,18100
3,000181,2025-10-10 12:44:23.316556+00:00,131800
4,0002D8,2025-10-07 20:56:23.316556+00:00,23600
...,...,...,...
262783,FFFEF3,2025-09-16 06:39:23.316556+00:00,56000
262784,FFFF2C,2025-10-08 05:11:23.316556+00:00,10800
262785,FFFF43,2025-09-19 19:58:23.316556+00:00,78500
262786,FFFFA8,2025-10-08 04:01:23.316556+00:00,28800


In [71]:
import sqlite3

conn = sqlite3.connect("travel2.sqlite")
cursor = conn.cursor()

# Make some hotels share city names with airports_data.city for joining possibilities
# cursor.execute("UPDATE hotels SET location='Moscow' WHERE id IN (1,2)")
# cursor.execute("UPDATE hotels SET location='London' WHERE id IN (3,4)")
# cursor.execute("UPDATE hotels SET location='New York' WHERE id IN (5,6)")

# Ensure trip_recommendations cities match some hotel cities
# cursor.execute("UPDATE trip_recommendations SET location='Moscow' WHERE id IN (1,2)")
# cursor.execute("UPDATE trip_recommendations SET location='London' WHERE id IN (3,4)")
# cursor.execute("UPDATE trip_recommendations SET location='New York' WHERE id IN (5,6)")

# cursor.execute("UPDATE car_rentals SET location='Moscow' WHERE id IN (1,2)")
# cursor.execute("UPDATE car_rentals SET location='London' WHERE id IN (3,4)")
# cursor.execute("UPDATE car_rentals SET location='New York' WHERE id IN (5,6)")

conn.commit()
# conn.close()


In [72]:
pd.read_sql(f"SELECT * from hotels", conn)

,id,name,location,price_tier,checkin_date,checkout_date,booked
0,1,Hilton Basel,Basel,Luxury,2024-04-22,2024-04-20,1
1,2,Marriott Zurich,Zurich,Upscale,2024-04-14,2024-04-21,1
2,3,Hyatt Regency Basel,Basel,Upper Upscale,2024-04-02,2024-04-20,1
3,4,Radisson Blu Lucerne,Lucerne,Midscale,2024-04-24,2024-04-05,1
4,5,Best Western Bern,Bern,Upper Midscale,2024-04-23,2024-04-01,1
5,6,InterContinental Geneva,Geneva,Luxury,2024-04-23,2024-04-28,1
6,7,Sheraton Zurich,Zurich,Upper Upscale,2024-04-27,2024-04-02,0
7,8,Holiday Inn Basel,Basel,Upper Midscale,2024-04-24,2024-04-09,0
8,9,Courtyard Zurich,Zurich,Upscale,2024-04-03,2024-04-13,0
9,10,Comfort Inn Bern,Bern,Midscale,2024-04-04,2024-04-16,0


In [73]:
pd.read_sql(f"SELECT * from trip_recommendations", conn)

,id,name,location,keywords,details,booked
0,1,Basel Minster,Basel,"landmark, history","Visit the historic Basel Minster, a beautiful ...",1
1,2,Kunstmuseum Basel,Basel,"art, museum",Explore the extensive art collection at the Ku...,1
2,3,Zurich Old Town,Zurich,"history, architecture",Take a stroll through the charming streets of ...,1
3,4,Lucerne Chapel Bridge,Lucerne,"landmark, history",Walk across the iconic Chapel Bridge in Lucerne.,1
4,5,Bern Bear Park,Bern,"wildlife, park",Visit the Bern Bear Park and see the city's fa...,1
5,6,Lake Geneva,Geneva,"nature, lake",Enjoy stunning views of Lake Geneva and the su...,1
6,7,Swiss National Museum,Zurich,"history, museum",Learn about Swiss history and culture at the S...,0
7,8,Basel Zoo,Basel,"wildlife, zoo",Spend a day exploring the diverse animal exhib...,0
8,9,Mount Pilatus,Lucerne,"nature, mountain",Take a cable car ride to the top of Mount Pila...,0
9,10,Bern Cathedral,Bern,"landmark, history","Visit the impressive Bern Cathedral, a stunnin...",0


In [74]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

question = "What is the average, minimum, and maximum age for all French musicians?"
schema = """
   "stadium" "Stadium_ID" int , "Location" text , "Name" text , "Capacity" int , "Highest" int , "Lowest" int , "Average" int , foreign_key:  primary key: "Stadium_ID" [SEP] "singer" "Singer_ID" int , "Name" text , "Country" text , "Song_Name" text , "Song_release_year" text , "Age" int , "Is_male" bool , foreign_key:  primary key: "Singer_ID" [SEP] "concert" "concert_ID" int , "concert_Name" text , "Theme" text , "Year" text , foreign_key: "Stadium_ID" text from "stadium" "Stadium_ID" , primary key: "concert_ID" [SEP] "singer_in_concert"  foreign_key: "concert_ID" int from "concert" "concert_ID" , "Singer_ID" text from "singer" "Singer_ID" , primary key: "concert_ID" "Singer_ID"
"""

input_text = " ".join(["Question: ",question, "Schema:", schema])

model_inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**model_inputs, max_length=512)

output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("SQL Query:")
print(output_text)


SQL Query:
["SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France'"]


In [75]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

MODEL_PATH = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Function to generate SQL dynamically
def generate_sql(schema_description: str, question: str) -> str:
    """
    Generate SQL from natural language question using T5 model.
    """
    input_text = f"Schema: {schema_description}\nQuestion: {question}"

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
        )

    sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sql = sql.strip().replace("\n", " ")

    if sql.lower().startswith("sql"):
        sql = sql.split(":", 1)[-1].strip()

    return sql


def run_query(sql_query: str):
    """
    Execute the generated SQL on the SQLite database.
    Returns DataFrame of results or error message.
    """
    conn = sqlite3.connect(local_file)
    try:
        df = pd.read_sql_query(sql_query, conn)
        return df
    except Exception as e:
        return f"Error executing SQL: {e}"
    finally:
        conn.close()


In [76]:
!pip install import-ipynb

Defaulting to user installation because normal site-packages is not writeable
